In [5]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = 'y')

he_init = tf.contrib.layers.variance_scaling_initializer()

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name = 'hidden1', kernel_initializer = he_init, activation = tf.nn.elu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name = 'hidden2', activation = tf.nn.elu, kernel_initializer = he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name = 'output')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')
    loss_summary = tf.summary.scalar('log_loss', loss)

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [3]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = 'tf_logs'
    if prefix:
        prefix += '-'
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

logdir = log_dir("mnist_dnn")
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [4]:
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_valid = mnist.validation.images
y_valid = mnist.validation.labels
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

m, n = X_train.shape

n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

checkpoint_path = '/tmp/my_deep_mnist_model.ckpt'
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_deep_mnist_model"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, 'rb') as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X:X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict = {X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("Epoch:", epoch,
             "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
             "\tLoss: {:.5f}%".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early stopping")
                    break

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Training was interrupted. Continuing at epoch 36
INFO:tensorflow:Restoring parameters from /tmp/my_deep_mnist_model.ckpt
Epoch: 40 	Validation accuracy: 97.400% 	Loss: 0.08493%
Epoch: 45 	Validation accuracy: 97.560% 	Loss: 0.08090%
Epoch: 50 	Validation accuracy: 97.740% 	Loss: 0.07717%
Epoch: 55 	Validation accuracy: 97.700% 	Loss: 0.07600%
Epoch: 60 	Validation accuracy: 97.800% 	Loss: 0.07419%
Epoch: 65 	Validation accuracy: 97.880% 	Loss: 0.07172%
Epoch: 70 	Validation accuracy: 97.760% 	Loss: 0.07327%
Epoch: 75 	Validation accuracy: 97.880% 	Loss: 0.06958%
Epoch: 80 	Validation accuracy: 97.940% 	Loss: 0.07082%
Epoch: 85 	Validation accuracy: 97.980% 	Loss: 0.06944%
Epoch: 90 	Validation accuracy: 98.040% 	Loss: 0.06894%
Epoch: 95 	Validation accuracy: 97.960% 	Loss: 0.06959%
Epoch: 100 	Valid

In [6]:
os.remove(checkpoint_epoch_path)
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})

print(accuracy_val)

INFO:tensorflow:Restoring parameters from ./my_deep_mnist_model
0.9777


In [10]:
he_init = tf.contrib.layers.variance_scaling_initializer()

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = 'y')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name = 'hidden1', kernel_initializer = he_init, activation = tf.nn.elu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name = 'hidden2', activation = tf.nn.elu, kernel_initializer = he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name = 'output')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')
    loss_summary = tf.summary.scalar('log_loss', loss)

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [12]:
checkpoint_path = '/tmp/my_deep_mnist_model_1.ckpt'
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_deep_mnist_model_1"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, 'rb') as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X:X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict = {X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("Epoch:", epoch,
             "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
             "\tLoss: {:.5f}%".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early stopping")
                    break

Epoch: 0 	Validation accuracy: 90.400% 	Loss: 0.34444%
Epoch: 5 	Validation accuracy: 94.160% 	Loss: 0.21409%
Epoch: 10 	Validation accuracy: 95.460% 	Loss: 0.16504%
Epoch: 15 	Validation accuracy: 96.280% 	Loss: 0.13852%
Epoch: 20 	Validation accuracy: 96.620% 	Loss: 0.11892%
Epoch: 25 	Validation accuracy: 96.940% 	Loss: 0.10448%
Epoch: 30 	Validation accuracy: 97.060% 	Loss: 0.09577%
Epoch: 35 	Validation accuracy: 97.320% 	Loss: 0.08934%
Epoch: 40 	Validation accuracy: 97.480% 	Loss: 0.08419%
Epoch: 45 	Validation accuracy: 97.540% 	Loss: 0.08019%
Epoch: 50 	Validation accuracy: 97.580% 	Loss: 0.07623%
Epoch: 55 	Validation accuracy: 97.740% 	Loss: 0.07488%
Epoch: 60 	Validation accuracy: 97.700% 	Loss: 0.07315%
Epoch: 65 	Validation accuracy: 97.820% 	Loss: 0.07107%
Epoch: 70 	Validation accuracy: 97.800% 	Loss: 0.07305%
Epoch: 75 	Validation accuracy: 97.920% 	Loss: 0.06951%
Epoch: 80 	Validation accuracy: 97.840% 	Loss: 0.07029%
Epoch: 85 	Validation accuracy: 97.840% 	Loss: 0.0

In [18]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy = accuracy.eval(feed_dict = {X: X_test, y: y_test})
    
print(accuracy)

INFO:tensorflow:Restoring parameters from ./my_deep_mnist_model_1
0.9769
